## Without web search

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano"
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

In [4]:
print(response['messages'][-1].content)

Short answer: I don’t have live internet access by default, and my training data covers information up to roughly late 2023 (with some topics having small updates earlier in 2024). I may not know about events, publications, or developments that happened after that unless you provide details or I’m using a browsing tool.

If you need the very latest info, you can:
- enable a browse/tool mode (if available here) and I’ll fetch current sources, or
- tell me the topic and date range and I’ll work from what I know and flag any potential gaps.

If you have a specific topic in mind, I can tailor my answer and note where post-cutoff updates might apply.


In [7]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='How up to date is your training knowledge?', additional_kwargs={}, response_metadata={}, id='1c8c2010-7273-419d-bbd7-3fa0cd6a358a'),
              AIMessage(content='Short answer: I don’t have live internet access by default, and my training data covers information up to roughly late 2023 (with some topics having small updates earlier in 2024). I may not know about events, publications, or developments that happened after that unless you provide details or I’m using a browsing tool.\n\nIf you need the very latest info, you can:\n- enable a browse/tool mode (if available here) and I’ll fetch current sources, or\n- tell me the topic and date range and I’ll work from what I know and flag any potential gaps.\n\nIf you have a specific topic in mind, I can tailor my answer and note where post-cutoff updates might apply.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1304, 'prompt_tokens': 15, 'total_tokens': 

In [12]:
# Example to compare the answer Agent answer with chat model
from langchain.chat_models import init_chat_model

chat_model = init_chat_model(
    model="gpt-5-nano")

response = chat_model.invoke(["How up to date is your training knowledge"])
                             
print(response.content)

My training data goes up to June 2024. I don’t have real-time internet access by default, so I may not know about events or developments that happened after that date. If your platform has browsing enabled, I can look up current information; otherwise, I can help interpret or analyze information you provide and explain concepts up to that cutoff. If you need the latest specifics, tell me the topic and I’ll suggest how to verify it with current sources.


## My question to CoPilot
Why does the agentic answer in cell 3 and 4 say that the model was trained with info up to late 2023 and the chat model answer in later cells says June 2024?

## Answer
The discrepancy you're seeing is because the two models (used by the agent and the chat model) have different knowledge cutoffs based on when their training data ends. This is common with different model versions or configurations:

The agentic answer (from the model used in the agent setup) reports a cutoff of "late 2023" (with some updates into early 2024).
The chat model answer reports a cutoff of "June 2024".
Even though the notebook code specifies "gpt-5-nano" for both, in practice, this could resolve to different underlying models or versions depending on your environment/configuration (e.g., one might be an older GPT-4 variant, while the other is a newer GPT-4o or similar). Model providers often release updates with extended cutoffs, and different instances or API calls might use different versions.

If this is intentional for the demo, it highlights how agents can use tools to access real-time data beyond their base training cutoff, while direct model queries are limited to their trained knowledge. If you want consistent cutoffs, ensure both use the exact same model version in your setup.

In [ ]:
# Underlying model of the chat
print("Chat model name:", chat_model.model_name)
print("Chat model details:", chat_model)

# Underlying model of the agent in the graph structure
graph = agent.get_graph()
print("Graph nodes:", list(graph.nodes.keys()))
print("Graph edges:", list(graph.edges))

# Inspect Individual Nodes
model_name_in_graph = None
for node_name, node in graph.nodes.items():
    print(f"\nNode: {node_name}")
    if hasattr(node, '__dict__'):
        pprint(vars(node))  # Print all attributes of the node
    else:
        print(f"Node object: {node}")
    if hasattr(node, 'runnable'):
        print(f"Runnable: {node.runnable}")
        if hasattr(node.runnable, 'model_name'):
            print(f"Model name: {node.runnable.model_name}")
            if node_name == 'model':
                model_name_in_graph = node.runnable.model_name

# Since the agent was created with create_agent, it should have stored config
print("Agent config:", getattr(agent, 'config', 'No config attribute'))
print("Agent attributes:", dir(agent))

# If the model name is still not showing in the Mermaid diagram, it means the underlying model
# in the agent is not directly accessible through these attributes.

Chat model name: gpt-5-nano
Chat model details: profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x78df982035c0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x78df98203290> root_client=<openai.OpenAI object at 0x78df982011c0> root_async_client=<openai.AsyncOpenAI object at 0x78df982035f0> model_name='gpt-5-nano' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True
Graph nodes: ['__start__', 'model', '__end__']
Graph edges: [Edge(source='__start__', target='model', data=None, conditional=False), Edge(sou

## Add web search tool

In [ ]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

In [ ]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)

In [ ]:
from pprint import pprint

pprint(response['messages'])

trace: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r